<a href="https://colab.research.google.com/github/Anshuman-02905/URBAN_MONITORING_SYSTEM/blob/main/EnhancedDataAcquisition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install geemap
!pip install geopandas
!apt-get update
!apt-get install libgdal-dev -y
!apt-get install python-gdal -y
!apt-get install python-numpy python-scipy -y

In [2]:
import json
import geemap
import os
import glob
import gdal ## fingers crossed!
import os
import ee
import pandas as pd
import numpy as np
import math
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, Polygon
import folium
import sys


In [3]:
Map=geemap.Map()

In [4]:
left_dist=0.1033789690619983
right_dist=0.1033789690619983
upper_dist=0.1402878671830234
lower_dist=0.1402878671830234

In [5]:
root="/content/drive/MyDrive/NEW_DATASET/"
Sentinel_tif=root+"Sentinel_tif/" 
Sentinel_png=root+"Sentinel_png/"
urban_tif=root+"Urban_tif/"
urban_png=root+"Urban_png/"
temporal_png=root+"Temporal_png/"
temporal_tif=root+"Temporal_tif/"
filename=root+"ACTIVE_LOG.json"
dirs=[root,Sentinel_tif,Sentinel_png,urban_tif,urban_png,temporal_tif,temporal_png]
path = [Sentinel_tif,urban_tif,temporal_tif]

for i in dirs:
  if os.path.exists(i)==False:
    os.mkdir(i)

In [6]:
dir=[Sentinel_tif,Sentinel_png,urban_tif,urban_png,temporal_tif,temporal_png]
for i in dir:
  dir = os.listdir(i)  
  # Checking if the list is empty or not
  if len(dir) != 0:
    files = glob.glob(i+"*")
    for f in files:
      os.remove(f)

In [7]:
res=get_final_list(72.72114537234593,23.96269296245415,)
result=get_boundaries(res,0)

NameError: ignored

In [8]:
def get_boundaries(final_list,c):
  try:
    coordinates = final_list
    def create_polygon(coords, polygon_name):
      ''' Create a polygon from coordinates'''
      polygon = Polygon(coordinates)
      gdf = gpd.GeoDataFrame(crs = {'init' :'epsg:4326'})
      gdf.loc[0,'name'] = polygon_name
      gdf.loc [0, 'geometry'] = polygon
      return gdf
    shapefile = create_polygon(coordinates, "ROI")
    shapefile.to_file("newboundary.shp")
    roi_shp="/content/newboundary.shp"
    roi_shp = geemap.shp_to_ee(roi_shp)
    return get_image(roi_shp,c)
  except Exception as e:
    print(e)
    return 0

In [9]:
def update_error(c,final_list_string,error):
  data=dict()
  log_p=r"/content/drive/MyDrive/NEW_DATASET/LOG.txt"
  f = open(log_p, "a")
  f.write("EXCEPTION"+str(c)+final_list_string+"\n")
  f.close()
  error+=1
  with open(filename) as json_file:
   data = json.load(json_file)
   data["Error_num"]=error
  with open(filename, 'w') as file_object:  #open the file in write mode
   json.dump(data, file_object)

In [10]:
def get_image(roi,c):
  try:
    sentinel = ee.ImageCollection('COPERNICUS/S2_SR').select(['B2','B4', 'B3', 'B11','B8']) \
                  .filterDate('2017-01-01', '2020-01-30') \
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',5)) \
                  .filterBounds(roi).mean()
    sentinel_rgb = sentinel.divide(10000).clip(roi).select(['B4', 'B3', 'B2']).unmask()
    visualization_rgb = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
    }
    Map.addLayer(sentinel_rgb.clip(roi), visualization_rgb, 'RGB')
    Map.centerObject(roi)
    mask = ee.Image("DLR/WSF/WSF2015/v1").clip(roi)
    visualization_mask = {
    'min': 0,
    'max': 255, 
    }
    mask=mask.expression('b("settlement") == 255 ? 255:0',{'settlement':mask.select('settlement')}).clip(roi)
    Map.addLayer(mask.clip(roi), visualization_mask, "Human2")



    NDVI = sentinel.normalizedDifference(['B8', 'B4']).rename(['NDVI']);
    MNDWI = sentinel.normalizedDifference(['B3', 'B11']).rename(['MNDWI']); 
    NDBI = sentinel.normalizedDifference(['B11', 'B8']).rename(['NDBI'])
    sentinel=sentinel.addBands(NDBI).addBands(MNDWI).addBands(NDVI)
    image_temporal_mean=sentinel.clip(roi).select(['NDVI','NDBI','MNDWI']).unmask()
    temporal_mean_vis={'bands':['NDBI','NDVI','MNDWI'],'min':-1,'max':1}
    Map.addLayer(image_temporal_mean.clip(roi), temporal_mean_vis, "Temporal_mean Color")
    return download_image(sentinel_rgb,mask,image_temporal_mean,roi,c)
  except Exception as e:
    print("Error in get_image")
    print(e)
    return -1

In [11]:
def get_final_list(Lon,Lat):
  try:
    LL=[Lon,Lat]
    UL=[Lon,Lat+left_dist]
    LR=[Lon+lower_dist,Lat]
    UR=[Lon+lower_dist,Lat+left_dist]
    final_list=[LL,LR,UR,UL,LL]
    return final_list
  except Exception as e: 
    print(e)
    print("Error in get_final_list Image")
    return -1

In [12]:
def download_image(RGB,MASK,TEMPORAL,roi,c):
    try:
      urban=urban_tif+str(c)+".tif"
      rgb=Sentinel_tif+str(c)+".tif"
      temporal=temporal_tif+str(c)+".tif"
      geemap.ee_export_image(MASK, filename=urban, scale=10, region=roi.geometry(), file_per_band=False)
      geemap.ee_export_image(RGB, filename=rgb, scale=10, region=roi.geometry(), file_per_band=False)
      geemap.ee_export_image(TEMPORAL, filename=temporal, scale=10, region=roi.geometry(), file_per_band=False)
      return convert_image(c)
    except Exception as e: 
      print(e)
      print("Error in download_image Image")
      return -1
    

    

In [13]:
def convert_image(c):
    urban=urban_tif+str(c)+".tif"
    rgb=Sentinel_tif+str(c)+".tif"
    temporal=temporal_tif+str(c)+".tif"
    urban_download=urban_png+str(c)+".png"
    rgb_download=Sentinel_png+str(c)+".png"
    temporal_download=temporal_png+str(c)+".png"
    options_list_3band = ['-ot Byte','-of PNG','-scale 0 0.3 0 255'] 
    options_list_tempband = ['-ot Byte','-of PNG','-scale -0.6 0.6 0 255'] 

    options_list_1band = ['-ot Byte','-of PNG']
    options_string_RGB = " ".join(options_list_3band)
    options_string_temp=" ".join(options_list_tempband)
    options_string = " ".join(options_list_1band)

    gdal.Translate(urban_download,urban, options=options_string)
    gdal.Translate(rgb_download,rgb, options=options_string_RGB)
    gdal.Translate(temporal_download,temporal, options=options_string_temp)
    xml1=urban_download+".aux.xml"
    xml2=rgb_download+".aux.xml"
    xml3=temporal_download+".aux.xml"

    os.remove(xml1)
    os.remove(xml2)
    os.remove(xml3)
    os.remove(urban)
    os.remove(rgb)
    os.remove(temporal)
    return 1


In [43]:
convert_image(0)

1

In [ ]:

#update_error(0,final_list_string,0)
result=1


In [14]:
final_list=list()
count=0

#cord=[23.96269296245415, 72.72114537234593]
cord=[23.96269296245415, 86.72114537234593]



data=dict()

with open(filename) as json_file:
    data = json.load(json_file)
Lat=data["Active_latitude"]
i=data["Active_I"]
error=data['Error_num']
c=data["Lat_c"]
Lon=cord[1]

for j in range(i,100):
  Lon=cord[1]
  data=dict()
  with open(filename) as json_file:
      data = json.load(json_file)
  data["Active_latitude"]=Lat
  data["Lat_c"]=c
  data['Active_I']=j
  with open(filename, 'w') as file_object:  #open the file in write mode
   json.dump(data, file_object)

  for j in range(0,100):
    print(Lon,Lat)
    with open(filename) as json_file:
     data = json.load(json_file)
     error=data['Error_num']
    print("Image_number",c)
    final_list=get_final_list(Lon,Lat)
    final_list_string = ' '.join([str(elem) for elem in final_list])
    num=str(c)
    f = open(r"/content/drive/MyDrive/NEW_DATASET/LOG.txt", "a")
    f.write(num+"\n"+final_list_string+"\n")
    f.close()
    print(final_list)
    result=get_boundaries(final_list,c)
    #result=result*-1
    if result==-1: 
     update_error(c,final_list_string,error)

    if result!=-1:
      c+=1


    # Getting the list of directories
    for i in path:
      dir = os.listdir(i)  
      # Checking if the list is empty or not
      if len(dir) != 0:
        files = glob.glob(i+"*")
        for f in files:
          os.remove(f)

    Lon+=lower_dist
  Lat+=left_dist

86.72114537234593 23.96269296245415
Image_number 0
[[86.72114537234593, 23.96269296245415], [86.86143323952895, 23.96269296245415], [86.86143323952895, 24.06607193151615], [86.72114537234593, 24.06607193151615], [86.72114537234593, 23.96269296245415]]


/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: Assigning CRS to a GeoDataFrame without a geometry column is now deprecated and will not be supported in the future.
  import sys


Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/NEW_DATASET/Urban_tif/0.tif
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/NEW_DATASET/Sentinel_tif/0.tif
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/NEW_DATASET/Temporal_tif/0.tif
86.86143323952895 23.96269296245415
Image_number 1
[[86.86143323952895, 23.96269296245415], [87.00172110671197, 23.96269296245415], [87.00172110671197, 24.06607193151615], [86.86143323952895, 24.06607193151615], [86.86143323952895, 23.96269296245415]]


/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: Assigning CRS to a GeoDataFrame without a geometry column is now deprecated and will not be supported in the future.
  import sys


Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/NEW_DATASET/Urban_tif/1.tif
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/NEW_DATASET/Sentinel_tif/1.tif
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/NEW_DATASET/Temporal_tif/1.tif
87.00172110671197 23.96269296245415
Image_number 2
[[87.00172110671197, 23.96269296245415], [87.14200897389499, 23.96269296245415], [87.14200897389499, 24.06607193151615], [87.00172110671197, 24.06607193151615], [87.00172110671197, 23.96269296245415]]


...

/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: Assigning CRS to a GeoDataFrame without a geometry column is now deprecated and will not be supported in the future.
  import sys


Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/NEW_DATASET/Urban_tif/6.tif
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/NEW_DATASET/Sentinel_tif/6.tif
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/NEW_DATASET/Temporal_tif/6.tif
87.70316044262707 23.96269296245415
Image_number 7
[[87.70316044262707, 23.96269296245415], [87.84344830981009, 23.96269296245415], [87.84344830981009, 24.06607193151615], [87.70316044262707, 24.06607193151615], [87.70316044262707, 23.96269296245415]]


/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: Assigning CRS to a GeoDataFrame without a geometry column is now deprecated and will not be supported in the future.
  import sys


Generating URL ...
Please wait ...


KeyboardInterrupt: ignored

In [ ]:
Map

Point (77.3643, 12.8805) at 611m/px
RGB: Image (3 bands)
  B2: None
  B3: None
  B4: None
Human2: Image (1 band)
  constant: None
Temporal_mean Color: Image (3 bands)
  MNDWI: None
  NDBI: None
  NDVI: None


In [ ]:
Map

Map(center=[72.77283485835568, 24.032836896045662], controls=(WidgetControl(options=['position', 'transparent_…